In [1]:
import os
#os.chdir('C:\\Users\\Administrator\\Desktop\\Prudhvi\\Prudhvi\\Data Science\\Data')
os.chdir('C:\\Users\\Administrator\\Desktop\\Prudhvi\\Prudhvi\\Data Science\\Data')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

Reading the file

In [2]:
pd.options.display.max_columns = None
#Home
#telco=pd.read_csv('Telco_Customer_Attr.csv')
#Office
telco=pd.read_csv('Teleco_Cust_Attr.csv')
telco.head()
#print(telco.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Checking the columns if any othem contains null or not

In [3]:
telco.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges         True
Churn               False
dtype: bool

Fixing the missing values using the Stratified mean value imputation

In [4]:
Mean_No_TotalCharges=telco.groupby('Churn')['TotalCharges'].mean()['No']
Mean_Yes_TotalCharges=telco.groupby('Churn')['TotalCharges'].mean()['Yes']

In [5]:
telco.loc[(telco['TotalCharges'].isnull()) & (telco['Churn']=='No'),['TotalCharges']]=Mean_No_TotalCharges
telco.loc[(telco['TotalCharges'].isnull()) & (telco['Churn']=='Yes'),['TotalCharges']]=Mean_Yes_TotalCharges

Dropping the columns which dont explain much variance

In [6]:
telco.drop(['customerID'],axis=1,inplace=True)

In [7]:
telco.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [8]:
telco.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Data type conversion

In [9]:
telco['SeniorCitizen']=telco['SeniorCitizen'].astype('object')

In [10]:
telco_backup=telco

In [11]:
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
encoding=defaultdict(LabelBinarizer)
columns_nominal=['gender','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','PaymentMethod']
for i in columns_nominal:
    encoded_df=pd.DataFrame()
    d=pd.DataFrame(encoding[i].fit_transform(telco[i]))
    test_column=d.columns.values
    list1=[i+'_'+str(j) for j in test_column]
    d.columns=list1
    encoded_df=pd.concat([encoded_df,d],axis=1)
    telco.drop(i,axis=1, inplace=True)
    telco=pd.concat([telco,encoded_df],axis=1)

In [12]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
telco['Contract']=lb_make.fit_transform(telco["Contract"])

In [13]:
telco.head()

,SeniorCitizen,tenure,Contract,MonthlyCharges,TotalCharges,Churn,gender_0,Partner_0,Dependents_0,PhoneService_0,MultipleLines_0,MultipleLines_1,MultipleLines_2,InternetService_0,InternetService_1,InternetService_2,OnlineSecurity_0,OnlineSecurity_1,OnlineSecurity_2,OnlineBackup_0,OnlineBackup_1,OnlineBackup_2,DeviceProtection_0,DeviceProtection_1,DeviceProtection_2,TechSupport_0,TechSupport_1,TechSupport_2,StreamingTV_0,StreamingTV_1,StreamingTV_2,StreamingMovies_0,StreamingMovies_1,StreamingMovies_2,PaperlessBilling_0,PaymentMethod_0,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
0,0,1,0,29.85,29.85,No,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
1,0,34,1,56.95,1889.50,No,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,1
2,0,2,0,53.85,108.15,Yes,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1
3,0,45,1,42.30,1840.75,No,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0
4,0,2,0,70.70,151.65,Yes,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0


from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
#enc.fit(telco['gender_lb'])
enc.fit(telco['MultipleLines_LE'].values.reshape(-1,1))
X=enc.transform(telco['MultipleLines_LE'].values.reshape(-1,1)).toarray()


In [14]:
y=telco['Churn']
telco=telco.drop('Churn',axis=1)

#### Train test split

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(telco,y,test_size=0.30,random_state=42)

#### Applying  SVC Using GridSearchCV

In [16]:
from sklearn.svm import SVC
SVC_clf=SVC()